In [89]:
import pandas as pd
import numpy as np
import scipy as sp
import pulp
from scipy.optimize import linprog
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable, LpMinimize

In [90]:
# Assuming FoodDatabase.csv is in the same directory as your Jupyter Notebook
file_path = 'FoodDatabase.csv'
#All numbers (macros and micros) in the table are a serving of 100g

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Filter rows with nonzero 'Price / kg'
df_filtered = df[df['Price / kg'] != 0]

# Add 'Cost ($)' column with initial value of zero at the start
df_filtered.insert(0, 'Cost ($)', 0)

# Add Quantities' (g)' column with initial value of zero as the second column
df_filtered.insert(1, 'Quantities (g)', 0)

# Display the first few rows of the DataFrame to verify it was read correctly
df_filtered.info()
df_filtered.head()


#We want to minimize c_1 * x_1 + c_2 * x_2 ... c_n * x_n , where c is the cost per unit mass of a given food, and x is the mass of the given food (aka the objective function)

# The constraints are the macros and micros of a diet, and other conditions we set and must be met. 

#Excel was using Simplex LP method


<class 'pandas.core.frame.DataFrame'>
Index: 26 entries, 187 to 14168
Columns: 106 entries, Cost ($) to Comments
dtypes: float64(85), int64(15), object(6)
memory usage: 21.7+ KB


,Cost ($),Quantities (g),ID,name,Food Group,Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),...,Caffeine (mg),Theobromine (mg),200 Calorie Weight (g),Sugar (g),Price / kg,Protein Quality,Product Name,Last Updated,Source,Comments
187,0,0,168263,Pork Fresh Loin Center Loin (Chops) Boneless S...,Meats,122.81,3.09,23.75,0.00,0.00,...,0,0,162.602,0.000,5.158811,1.0,Smithfield Fresh Pork Center Cut Loin Boneless...,01-Feb-24,https://www.walmart.com/ip/Smithfield-Fresh-Po...,0
482,0,0,168877,Rice White Long-Grain Regular Raw Enriched,Grains and Pasta,349.06,0.66,7.13,79.95,0.12,...,0,0,54.795,0.060,1.228000,0.0,"Great Value Long Grain Enriched Rice, 20 lb USA",01-Feb-24,https://www.walmart.com/ip/Great-Value-Lentils...,0
505,0,0,168894,Wheat Flour White All-Purpose Enriched Bleached,Grains and Pasta,344.58,0.98,10.33,76.31,0.27,...,0,0,54.945,0.000,1.080264,0.0,"Great Value All-Purpose Flour, 5LB Bag USA",24-Feb-24,https://www.walmart.com/ip/Great-Value-All-Pur...,0
516,0,0,169736,Pasta Dry Enriched,Grains and Pasta,351.63,1.51,13.04,74.67,2.67,...,0,0,53.908,0.350,1.999590,0.0,"Great Value Elbows, 48 oz PASTA USA",01-Feb-24,https://www.walmart.com/ip/Great-Value-Elbows-...,"not worth it in the US, dairy better, pricey"
525,0,0,170379,Broccoli,Vegetables,30.77,0.37,2.82,6.64,1.70,...,0,0,588.235,0.835,2.513267,1.0,"Great Value Frozen Broccoli Cuts, 32 oz Steama...",01-Feb-24,https://www.walmart.com/ip/Great-Value-Frozen-...,0


In [127]:
# Define the model (for maintenance macro + micros)
maintenance_model = LpProblem(name="Maintenance_Diet_Minimum_Cost", sense=LpMinimize)

# Define the decision variables (what we are solving for; quantities of each food)
x = {i: LpVariable(name=f"x_{i}", lowBound=0, upBound=10000) for i in range(1, len(df_filtered)+1)}

# Arrays per column extracted from table
dollars_per_g = df_filtered.iloc[:, 100].values/1000 # compute array of dollars per g for each food
name_per_food = df_filtered.iloc[:, 102].values # get array of the names of each food (not database name but the specific item from the store)
calories_per_g = df_filtered.iloc[:, 5].values/100 # get array of caloric values for each food
protein_per_g = df_filtered.iloc[:, 7].values/100 # get array of amount of protein for each food
net_carbs_per_g = df_filtered.iloc[:, 95].values/100 # get array of net carbs for each food (this is a computed column I had made; carbs -= fiber = net carbs, because we will use fiber as a separate constraint later)
fiber_per_g = df_filtered.iloc[:, 10].values/100
fat_per_g = df_filtered.iloc[:, 6].values/100
protein_quality = df_filtered.iloc[:, 101].values
sugar_per_g = df_filtered.iloc[:, 99].values/100 # uses the more accurate computed sugar column on column # 99 (ie doesn't count glucose)
satfat_per_g = df_filtered.iloc[:, 12].values/100

maintenance_model += lpSum(x[i] * dollars_per_g[i-1] for i in range(1, len(df_filtered)+1)) # Objective Function

#Defining Constraints
#TODO : PULL THESE FROM SOMEWHERE ELSE, MAYBE READ CSV THAT HAS BODYWEIGHT AND OTHER STATS AND COMPUTE THE MAINTENANCE AND OTHER NEEDS BASED ON THAT
maintenance_calories = 2937 # set maintenance calories
protein_minimum = 242 # set protein minimum
net_carb_minimum = 236 # set net carb minimum
fiber_lbound = 30 # lower bound for fiber
fiber_ubound = 60 # upper bound for fiber
fat_minimum = 82
protein_quality_minimum = 0.4 #this is a fraction ie 0.4 = 40%
sugar_maximum = 75
sat_fat_maximum = 0.5 # this is a fraction

#Calorie Constraint Eqn
maintenance_model += (lpSum(x[i] * calories_per_g[i-1] for i in range(1, len(df_filtered)+1)) == maintenance_calories, "Maintenance Calories Constraint")
#Protein Constraint Eqn
maintenance_model += (lpSum(x[i] * protein_per_g[i-1] for i in range(1, len(df_filtered)+1)) >= protein_minimum, "Protein Minimum Constraint")
#Carb Constraint Eqn
maintenance_model += (lpSum(x[i] * net_carbs_per_g[i-1] for i in range(1, len(df_filtered)+1)) >= net_carb_minimum, "Net Carb Minimum Constraint")
#Fiber Constraint Eqn
maintenance_model += (lpSum(x[i] * fiber_per_g[i-1] for i in range(1, len(df_filtered)+1)) >= fiber_lbound, "Fiber Lower Bound Constraint")
maintenance_model += (lpSum(x[i] * fiber_per_g[i-1] for i in range(1, len(df_filtered)+1)) <= fiber_ubound, "Fiber Upper Bound Constraint")
#Fat Constraint Eqn
maintenance_model += (lpSum(x[i] * fat_per_g[i-1] for i in range(1, len(df_filtered)+1)) >= fat_minimum, "Fat Minimum Constraint")
#Protein Quality Eqn
maintenance_model += (lpSum(x[i] * protein_quality[i-1]  * protein_per_g[i-1] for i in range(1, len(df_filtered)+1))/protein_minimum >= protein_quality_minimum, "Protein Quality Constraint")
#Sugar Constraint Eqn
maintenance_model += (lpSum(x[i] * sugar_per_g[i-1] for i in range(1, len(df_filtered)+1)) <= sugar_maximum, "Sugar Constraint")
#SatFat Constraint Eqn
#maintenance_model += (lpSum(x[i] * satfat_per_g[i-1] for i in range(1, len(df_filtered)+1))/ <= sat_fat_maximum, "SatFat Constraint") #Currently Fast and loose with fat rules. Sat fats no more than 50% of total fats. May change this in the future.








#display full model info (comment out when not needed)
#print(maintenance_model)


#array of name per food



# Solve the optimization problem
status = maintenance_model.solve()

# Get the results and format them
print(f"status: {maintenance_model.status}, {LpStatus[maintenance_model.status]}")
monthly_cost = round(maintenance_model.objective.value() * 30, 2)
objective_value = round(maintenance_model.objective.value(), 2)
print(f"TOTAL COST OF DAILY DIET CALCULATED: ${objective_value}")

# Calculate and print the monthly diet cost

print(f"TOTAL COST OF MONTHLY DIET CALCULATED: ${monthly_cost}")

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['Food Name', 'Quantity (g)'])

# List to collect the data for the DataFrame
results_list = []

# Add the decision variables to the DataFrame
for i, var in x.items():
    var_value = var.value()  # Get the value of the decision variable
    if var_value > 0:
        var_value_rounded = f"{int(round(var_value, 0))} g"  # Round to nearest whole number and add ' g'
        results_list.append({'Food Name': name_per_food[i-1], 'Quantity (g)': var_value_rounded})

# Convert the list to a DataFrame and concatenate with the results_df
results_df = pd.concat([results_df, pd.DataFrame(results_list)], ignore_index=True)

# Sort the DataFrame in descending order based on the 'Quantity (g)' column
results_df['Quantity (g)'] = results_df['Quantity (g)'].str.replace(' g', '').astype(int)  # Convert quantities to int for sorting
results_df = results_df.sort_values(by='Quantity (g)', ascending=False)
results_df['Quantity (g)'] = results_df['Quantity (g)'].astype(str) + ' g'  # Convert back to string with ' g'

# Display the results DataFrame
print("CONSISTING OF THE FOLLOWING FOODS:")
print(results_df)

# Print errors per constraint equation
print("\nError per constraint equation:")
for name, constraint in maintenance_model.constraints.items():
    print(f"{name}: ,{constraint.value()}")

status: 1, Optimal
TOTAL COST OF DAILY DIET CALCULATED: $3.39
TOTAL COST OF MONTHLY DIET CALCULATED: $101.73
CONSISTING OF THE FOLLOWING FOODS:
                                           Food Name Quantity (g)
1  Perdue, Fresh Chicken Gizzards (May Contain He...       1081 g
0         Great Value All-Purpose Flour, 5LB Bag USA        251 g
2                           Popcorn Kernels USA 2 lb        244 g

Error per constraint equation:
Maintenance_Calories_Constraint: ,-5.000001692678779e-07
Protein_Minimum_Constraint: ,2.8810000074486197e-06
Net_Carb_Minimum_Constraint: ,71.74999720099999
Fiber_Lower_Bound_Constraint: ,4.881439409999999
Fiber_Upper_Bound_Constraint: ,-25.11856059
Fat_Minimum_Constraint: ,-9.200000050668677e-08
Protein_Quality_Constraint: ,0.49269697469834717
Sugar_Constraint: ,-75.0


In [117]:
#Debugger cell
protein_quality = df_filtered.iloc[:, 101].values

print (protein_quality)


[1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1.
 1. 1.]
